In [71]:
import random

class Node():
    def __init__(self, left, right, is_leaf = False, idx = None):
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.idx = idx

        if not self.is_leaf:
            self.value = left.value + right.value

        self.parent = None

        if left is not None:
            left.parent = self
        if right is not None:
            right.parent = self

    @classmethod
    def create_leaf(cls, value, idx):
        leaf = cls(None, None, True, idx)
        leaf.value = value
        return leaf


class SumTree():
    def __init__(self, weights: list):
        nodes = [Node.create_leaf(v, i) for i,v in enumerate(weights)]
        self.leaf_nodes = nodes
        while len(nodes) > 1:
            inodes = iter(nodes)
            nodes = [Node(*pair) for pair in zip(inodes, inodes)]

        self.top_node = nodes[0]
        
    def retrieve(self, value: float, node: Node):
        if node.is_leaf:
            return node
        if node.left.value >= value:
            return self.retrieve(value, node.left)
        else:
            return self.retrieve(value - node.left.value, node.right)

    def update(self, idx, new_value: float):
        node = self.leaf_nodes[idx]
        change = new_value - node.value
        node.value = new_value
        self.propagate_changes(change, node.parent)

    def propagate_changes(self, change:float, node: Node):
        node.value += change
        if node.parent is not None:
            self.propagate_changes(change, node.parent)

    def draw_idx(self):
        u = random.uniform(0, self.top_node.value)
        return self.retrieve(u, self.top_node).idx


In [4]:
sumTree = SumTree([1,4,2,3])
sumTree.top_node.value

10

In [5]:
sumTree.update(3, 10)

In [6]:
sumTree.top_node.value

17

In [7]:
x = []
for i in range(1000000):
    x.append(sumTree.draw_idx())

sum([1 for i in x if i == 3]) / sum([1 for j in x if j == 0])

9.984864185367014

In [35]:
sumTree.draw_idx()

3

[1,
 3,
 1,
 1,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 2,
 1,
 1,
 3,
 0,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 0,
 3,
 3]

In [139]:
from typing import Tuple, List, Union
from collections import namedtuple, deque
import numpy as np

Experience = namedtuple("Experience", 
                            field_names = ["state", "action", "reward", "done", "next_state"])

class ReplayMemory:
    """
    Original Replay Memory by Lin. Used for vanilla DQN, no prioritized Replay or bootstrapping with n>1.
    Used to store and sample experiences
    """
    def __init__(self, capacity: int, epsilon = 0.01, alpha = 0.6, beta = 0.4) -> None:
        """
        Args: 
            capacity: size of buffer
        """
        self.buffer = deque(maxlen=capacity)
        self.sumTree = SumTree([0 for x in range(capacity)])
        self.epsilon = epsilon
        self.alpha = alpha
        self.beta = beta
        self.first_append = True

    def __len__(self):
        return len(self.buffer)

    def append(self, sample: Experience, tderror) -> None:
        """
        Append sample
        Args:
            sample: A sample of an experience to store. Experience is a tuple(state, action, reward, done, next_state)
        """
        self.buffer.append(sample)
        priority = self.get_priority(tderror)
        #print(priority)
        self.sumTree.update(len(self.buffer), priority)


    def get_priority(self, tderror):            
        pi = abs(tderror) + self.epsilon
        
        if self.first_append:
            Pi = (pi ** self.alpha) / pi
            self.first_append = False
        else:
            Pi = (pi ** self.alpha) / self.sumTree.top_node.value
        print(Pi)
        print(self.sumTree.top_node.value)
        print("-----")
        return Pi

    def sample(self, batch_size: int = 1) -> Tuple:
        """
        Return batch of buffer, randomly (uniformely).
        Args: 
            batch_size: size of batch
        """
        idxs = []
        for i in range(batch_size):
            idxs.append(self.sumTree.draw_idx())
        #[idxs.append(self.sumTree.draw_idx()) for _ in range(batch_size)]
        
        states, actions, rewards, dones, next_states = \
            zip(*[self.buffer[idx] for idx in idxs])

        #TODO IS_WEIGHTS

        return np.array(states), np.array(actions), \
            np.array(rewards, dtype=np.float32), np.array(dones, dtype=bool), \
            np.array(next_states)

# NOW LETS TEST IT

In [37]:
import numpy as np
import os
os.environ.setdefault('PATH', '')
from collections import deque
import gym
from gym import spaces


USE_PIL = True
if USE_PIL:
    # you should use pillow-simd, as it is faster than stardand Pillow
    from PIL import Image
else:
    import cv2
    cv2.ocl.setUseOpenCL(False)


class TimeLimit(gym.Wrapper):
    def __init__(self, env, max_episode_steps=None):
        super(TimeLimit, self).__init__(env)
        self._max_episode_steps = max_episode_steps
        self._elapsed_steps = 0

    def step(self, ac):
        observation, reward, done, info = self.env.step(ac)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self._max_episode_steps:
            done = True
            info['TimeLimit.truncated'] = True
        return observation, reward, done, info

    def reset(self, **kwargs):
        self._elapsed_steps = 0
        return self.env.reset(**kwargs)

class ClipActionsWrapper(gym.Wrapper):
    def step(self, action):
        import numpy as np
        action = np.nan_to_num(action)
        action = np.clip(action, self.action_space.low, self.action_space.high)
        return self.env.step(action)

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True, dict_space_key=None):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        If the environment uses dictionary observations, `dict_space_key` can be specified which indicates which
        observation should be warped.
        """
        super().__init__(env)
        self._width = width
        self._height = height
        self._grayscale = grayscale
        self._key = dict_space_key
        if self._grayscale:
            num_colors = 1
        else:
            num_colors = 3

        new_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(self._height, self._width, num_colors),
            dtype=np.uint8,
        )
        if self._key is None:
            original_space = self.observation_space
            self.observation_space = new_space
        else:
            original_space = self.observation_space.spaces[self._key]
            self.observation_space.spaces[self._key] = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3

    def observation(self, obs):
        if self._key is None:
            frame = obs
        else:
            frame = obs[self._key]
        if USE_PIL:
            frame = Image.fromarray(frame)
            if self._grayscale:
                frame = frame.convert("L")
            frame = frame.resize((self._width, self._height))
            frame = np.array(frame)
        else:
            if self._grayscale:
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            frame = cv2.resize(
                frame, (self._width, self._height),
                interpolation=cv2.INTER_AREA
            )
        if self._grayscale:
            frame = np.expand_dims(frame, -1)

        if self._key is None:
            obs = frame
        else:
            obs = obs.copy()
            obs[self._key] = frame
        return obs

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=0)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

    def count(self):
        frames = self._force()
        return frames.shape[1:frames.ndim]

    def frame(self, i):
        return self._force()[i, ...]


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0]*k, shp[1], shp[2]), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class SkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


def make_atari(env_id, max_episode_steps=None,
               skip_noop=False, skip_maxskip=False):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    if not skip_noop:
        env = NoopResetEnv(env, noop_max=30)
    if not skip_maxskip:
        env = MaxAndSkipEnv(env, skip=4)
    else:
        env = SkipEnv(env, skip=4)
    if max_episode_steps is not None:
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
    return env


class ImageToPyTorch(gym.ObservationWrapper):
    """
    Change image shape to CWH
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        new_shape = (old_shape[-1], old_shape[0], old_shape[1])
        self.observation_space = gym.spaces.Box(
            low=0.0, high=1.0, shape=new_shape, dtype=np.uint8)

    def observation(self, observation):
        return np.swapaxes(observation, 2, 0)


def wrap_deepmind(env, episode_life=True, clip_rewards=True,
                  frame_stack=False, scale=False, pytorch_img=False,
                  frame_stack_count=4, skip_firereset=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        if not skip_firereset:
            env = FireResetEnv(env)
    env = WarpFrame(env)
    if pytorch_img:
        env = ImageToPyTorch(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, frame_stack_count)
    return env

In [38]:
import torch
import torch.nn as nn
import numpy as np

class DQN(nn.Module):
    """
    Neural Network, choosing actions
    """
    def __init__(self, n_in, n_out):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(n_in[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(n_in)

        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_out)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        #x = x.float() #!!! not sure. I need this, because the images are now stored as uint8 and not float32 anymore
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)



In [132]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time
from datetime import datetime
import os
import resource


class PER_Agent():
    """
        Vanilla DQN Agent from Mnih2013, Mnih2015
    """
    def __init__(self, env, env_name, config):
        #Hyperparameters
        self.hp_replay_memory_capacity = config['hp_replay_memory_capacity']
        self.hp_gamma = config['hp_gamma']
        self.hp_epsilon_start = config['hp_epsilon_start']
        self.hp_epsilon_end = config['hp_epsilon_end']
        self.hp_epsilon_decay_last_frame = config['hp_epsilon_decay_last_frame']
        self.hp_learning_rate = config['hp_learning_rate']
        self.hp_replay_memory_start_after = config['hp_replay_memory_start_after']
        self.hp_batch_size = config['hp_batch_size']
        self.hp_target_update_after = config['hp_target_update_after']
        self.hp_update_frequency = config['hp_update_frequency']

        #Training loop: Episode length differs in each game, so timesteps / frames are better! (bellemare 2017)
        self.nr_of_total_frames = config['nr_of_total_frames'] 
        self.nr_of_evaluation_frames = config['nr_of_evaluation_frames']
        self.nr_of_frames_before_evaluation = config['nr_of_frames_before_evaluation']
        
        #Evaluation variables
        self.timesteps_overall = -1
        self.timesteps_after_last_episode = 0
        self.train_obtained_returns = []
        self.train_avg_returns = []
        self.eval_obtained_returns = []
        self.eval_counter = 0
        self.eval_epsilon = 0.05
        self.eval_state_samples = []

        self.env = env
        self.env_name = env_name
           
        self.replay_memory = ReplayMemory(self.hp_replay_memory_capacity)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
        self.training_start_timestamp = datetime.now(tz=None).strftime("%Y-%m-%d_%H-%M-%S")

        #Network
        self.policy_net = DuelingDQN(self.env.observation_space.shape, self.env.action_space.n).to(self.device)
        self.target_net = DuelingDQN(self.env.observation_space.shape, self.env.action_space.n).to(self.device)
        self._update_target_net()

        self._initialize_evaluation_state_samples_for_max_pred_q_vals(100)


    def _update_target_net(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())


    def _reset(self):
        """
            reset environment and reset the obtained return
        """
        self.state = self.env.reset()
        self.last_obtained_return = 0.0

    #DDQN! need to tidy up a bit TODO
    def _calc_loss(self, batch):
        """
            Calculate L1-Loss for given batch.
        """
        states, actions, rewards, dones, next_states = batch
    
        states_v = torch.from_numpy(states).to(self.device)
        next_states_v = torch.from_numpy(next_states).to(self.device)
        actions_v = torch.from_numpy(actions).to(self.device)
        rewards_v = torch.from_numpy(rewards).to(self.device)
        done_mask = torch.BoolTensor(dones).to(self.device)
        
        state_action_values = self.policy_net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
        
        with torch.no_grad():
            next_actions = self.policy_net(next_states_v).max(1)[1]
            next_state_values = self.target_net(next_states_v).gather(1, next_actions.unsqueeze(-1)).squeeze(-1)
            next_state_values[done_mask] = 0.0
        next_state_values = next_state_values.detach()
        expected_state_action_values = (next_state_values * self.hp_gamma) + rewards_v

        return F.smooth_l1_loss(state_action_values, expected_state_action_values)

    def _get_epsilon(self):
        """
            Get current value for epsilon
        """
        return max(self.hp_epsilon_end, self.hp_epsilon_start - self.timesteps_overall / self.hp_epsilon_decay_last_frame)


    def _select_action(self, eval):
        """
            Select action
        """
        epsilon = self._get_epsilon()
        if eval:
            epsilon = self.eval_epsilon

        with torch.no_grad():
            state = torch.tensor(np.array([self.state], copy=False)).to(self.device)
            q_vals = self.policy_net(state)

        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
            with torch.no_grad():
                action_ = torch.tensor([action]).to(self.device)
                q_val = q_vals.gather(1, action_.unsqueeze(-1)).squeeze(-1)
        else:
            with torch.no_grad():
                q_val, action = torch.max(q_vals, dim=1)
                action = int(action.item())
                del state

        #TODO TDERROR, 1 pass more through target-net :( The next_state is only known afterwards! 
        return action, q_val #qvals


    def _play_step(self, eval=False):
        """
            Play one step and return if episode ended
        """
        action, state_action_values = self._select_action(eval)
        
        next_state, reward, done, _ = self.env.step(action)

        self.last_obtained_return += reward

        if not eval:
            #calculate q_vals of maxa for next state
            #calculate tderror TODO
            with torch.no_grad():
                next_state_ = torch.tensor(np.array([next_state], copy=False)).to(self.device)
                next_state_values = self.target_net(next_state_).max(1)[0]
                td_error = state_action_values - reward + (self.hp_gamma * next_state_values)
                #print(td_error)
            exp = Experience(self.state, action, reward, done, next_state)
            self.replay_memory.append(exp, td_error.item())

        self.state = next_state
        del next_state
        return done


    def train_agent(self):
        """
            Train the Agent. 
        """
        writer = SummaryWriter(comment="_"+self.env_name)
        optimizer = optim.Adam(self.policy_net.parameters(), lr=self.hp_learning_rate)
        print ("Start Training on %s" % self.device)

        episode = 0
        while True:
            episode += 1
            self._reset()
            done = False
            ts_episode_started = time.time()

            while(not done):
                self.timesteps_overall += 1
                done = self._play_step()
                
                if len(self.replay_memory) < self.hp_replay_memory_start_after:
                    continue
                    

                #learn
                if self.timesteps_overall % self.hp_update_frequency == 0:
                    optimizer.zero_grad()
                    batch = self.replay_memory.sample(self.hp_batch_size)
                    loss = self._calc_loss(batch)
                    loss.backward()
                    #gradient clipping, like dueling DQN proposes
                    clipping_value = 10 
                    torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), clipping_value)
                    optimizer.step()

                if self.timesteps_overall % self.hp_target_update_after == 0:
                    self._update_target_net()


            speed = (self.timesteps_overall - self.timesteps_after_last_episode) / (time.time() - ts_episode_started)         
            ram_usage = int(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss) / 1024 / 1024

            self.timesteps_after_last_episode = self.timesteps_overall
            self.train_obtained_returns.append(self.last_obtained_return)
            self.train_avg_returns.append(np.mean(self.train_obtained_returns[-100:]))

            self._write_tensorboard(writer, speed, ram_usage)

            print("Episode %d completed, timesteps played: %d, return: %d, speed %f, epsilon %f" 
                % (episode, self.timesteps_overall, self.last_obtained_return, speed, self._get_epsilon()))
            print("Mean return of last 100 games: %f" % self.train_avg_returns[-1])
            print("Pytorch memory usage: %2f (gb)" % ram_usage)
            print("Size of Replay Memory: %d" % len(self.replay_memory))
            
            #evaluate every nr_of_frames_before_evaluation
            if self.timesteps_overall >= self.nr_of_frames_before_evaluation * (self.eval_counter+1):
                self._evaluate(writer)

            if self.timesteps_overall >= self.nr_of_total_frames:
                break

        writer.close()


    def _initialize_evaluation_state_samples_for_max_pred_q_vals(self, n):
        """
            Get samples for evaluation by sampling n states from random agent
        """
        self.env.reset()
        for i in range(n):
            with torch.no_grad():
                action = self.env.action_space.sample()
                next_state, reward, done, _ =  self.env.step(action)
                self.eval_state_samples.append(next_state)
                del next_state


    def _get_max_pred_q_vals(self):
        """
            Get max q-values for each state in evaluation_state_samples
        """
        with torch.no_grad():
            max_q_vals = []
            for state in self.eval_state_samples:
                state = torch.from_numpy(np.array([state], copy=False)).to(self.device)#.type(self.dtype)
                q_vals = self.policy_net(state)
                max_q_vals.append(q_vals.max().item())
            return np.mean(max_q_vals)


    def _save_model_snapshot(self, score):
        """
            Save model. For each Evaluation Metric, always save the best.
        """ 
        if not os.path.isdir("models/DQN/" + self.env_name + "/" +  self.training_start_timestamp):
            os.mkdir("models/DQN/%s/%s" % (self.env_name, self.training_start_timestamp))
        torch.save(self.policy_net.state_dict(), "models/DQN/%s/%s/snapshot_%d_score_%d.dat" 
                                % (self.env_name, self.training_start_timestamp, self.eval_counter, score))


    def _evaluate(self, writer):
        """
            Pause Training and evaluate agent by running environment like training, 
            but without actually training the model and decreased exploration.
        """
        _eval_iteration_returns = []
        _eval_iteration_max_q_vals = []

        print("***************** Start Evaluation *****************")
        evaluation_episode = 0
        evaluation_frames = 0
        while True:
            evaluation_episode += 1
            self._reset()
            done = False
            while(not done):
                evaluation_frames += 1
                done = self._play_step(eval=True) 

            _eval_iteration_returns.append(self.last_obtained_return)
            _eval_iteration_max_q_vals.append(self._get_max_pred_q_vals())
            print("Evaluation episode %d ended with return %d" %(evaluation_episode, self.last_obtained_return))      

            if evaluation_frames > self.nr_of_evaluation_frames:
                break

        _score_avg_returns = np.mean(_eval_iteration_returns)
        _score_max_q = np.mean(_eval_iteration_max_q_vals)

        print("Avg evaluation score: %f" % _score_avg_returns)
        print("***************** End Evaluation *****************")

        self.eval_counter += 1
        self._save_model_snapshot(_score_avg_returns)
        self.eval_obtained_returns.append(_score_avg_returns)

        writer.add_scalar('Evaluation/AvgTotalReturnPerEpisode', _score_avg_returns, self.eval_counter)
        writer.add_scalar('Evaluation/MaxPredQVals', _score_max_q, self.eval_counter)

    def _write_tensorboard(self, writer, speed, ram_usage):
        writer.add_scalar('Training/AvgTotalReturn', self.train_avg_returns[-1], self.timesteps_overall)
        writer.add_scalar('Training/ObtainedReturns', self.last_obtained_return, self.timesteps_overall)
        writer.add_scalar('Training/MaxPredQVals', self._get_max_pred_q_vals(), self.timesteps_overall)
        writer.add_scalar('Parameter/Epsilon', self._get_epsilon(), self.timesteps_overall)
        writer.add_scalar('Parameter/Speed', speed, self.timesteps_overall)
        writer.add_scalar('Parameter/MemoryUsage', ram_usage, self.timesteps_overall)
        writer.add_scalar('Parameter/ReplayMemorySize', len(self.replay_memory), self.timesteps_overall)

   



In [133]:
class DuelingDQN(nn.Module):
    def __init__(self, n_in, n_out):
        super(DuelingDQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(n_in[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(n_in)

        self.value_stream = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

        self.advantage_stream = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_out)
        )

        self.conv.apply(self.init_weights)
        self.value_stream.apply(self.init_weights)
        self.advantage_stream.apply(self.init_weights)

        self.dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor #here

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        x = x.type(self.dtype) #!!! not sure. I need this, because the images are now stored as uint8 and not float32 anymore
        conv_out = self.conv(x).view(x.size()[0], -1)
        values = self.value_stream(conv_out)
        advantages = self.advantage_stream(conv_out)
        q_vals = values + (advantages - advantages.mean())

        return q_vals

    def init_weights(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

In [140]:
#env_name = 'PongNoFrameskip-v4'
import json  
env_name = 'PongNoFrameskip-v4'
env = make_atari(env_name, skip_noop=False, skip_maxskip=True)
env = wrap_deepmind(env, pytorch_img=True, frame_stack=False)
config = json.loads('{\
    "test_config": \
        {\
            "hp_replay_memory_capacity": 100000,\
            "hp_gamma": 0.99,\
            "hp_epsilon_start": 1.0,\
            "hp_epsilon_end": 0.1,\
            "hp_epsilon_decay_last_frame": 100000,\
            "hp_learning_rate": 0.00001,\
            "hp_replay_memory_start_after": 10000,\
            "hp_batch_size":32,\
            "hp_target_update_after": 10000,\
            "nr_of_total_frames":  25000000,\
            "nr_of_evaluation_frames": 10000,\
            "nr_of_frames_before_evaluation": 50000,\
            "hp_update_frequency": 4\
        },\
    "config_mnih2013":\
        {\
            "hp_replay_memory_capacity": 1000000, \
            "hp_gamma": 0.99,\
            "hp_epsilon_start": 1.0,\
            "hp_epsilon_end": 0.1,\
            "hp_epsilon_decay_last_frame": 1e6,\
            "hp_learning_rate": 0.00001,\
            "hp_replay_memory_start_after": 50000,\
            "hp_batch_size":32,\
            "hp_target_update_after": 40000,\
            "nr_of_total_frames":  10e6,\
            "nr_of_evaluation_frames": 10e3,\
            "nr_of_frames_before_evaluation": 50e3,\
            "hp_update_frequency": 4\
        },\
    "config_mnih2015":\
        {\
            "hp_replay_memory_capacity": 1000000, \
            "hp_gamma": 0.99,\
            "hp_epsilon_start": 1.0,\
            "hp_epsilon_end": 0.1,\
            "hp_epsilon_decay_last_frame": 1e6,\
            "hp_learning_rate": 0.00001,\
            "hp_replay_memory_start_after": 50000,\
            "hp_batch_size":32,\
            "hp_target_update_after": 10000,\
            "nr_of_total_frames":  50e6,\
            "nr_of_evaluation_frames": 520e3,\
            "nr_of_frames_before_evaluation": 50e3,\
            "hp_update_frequency": 4\
        }\
}')

dqn = PER_Agent(env, env_name, config['test_config'])
dqn.train_agent()

Start Training on cuda
0.31808150778174005
0
-----
7.831204289189859
0.31808150778174005
-----
0.6893852388674514
8.149285796971599
-----
0.2859036435119703
8.83867103583905
-----
0.5610784079869724
9.12457467935102
-----
0.4836863297714358
9.685653087337993
-----
0.5722539581112602
10.169339417109429
-----
0.44370504036477665
10.741593375220688
-----
0.5311151552026265
11.185298415585464
-----
0.49874618346113536
11.716413570788092
-----
0.42130060043273265
12.215159754249227
-----
0.3355760084232688
12.63646035468196
-----
0.41233072361424034
12.972036363105229
-----
0.34123425157959264
13.384367086719468
-----
0.4122208014159609
13.725601338299061
-----
0.3143489086045308
14.137822139715022
-----
0.30906830208946995
14.452171048319553
-----
0.3022898364154292
14.761239350409022
-----
0.3070163396510295
15.063529186824452
-----
0.3030469150636286
15.370545526475482
-----
0.2888295489006427
15.67359244153911
-----
0.16812095810086708
15.962421990439752
-----
0.3546644077443775
16.1305

0.05923296569071286
43.80320651609708
-----
0.05989149318240915
43.86243948178779
-----
0.12874342403516809
43.9223309749702
-----
0.12894594221371772
44.05107439900537
-----
0.10684881029712813
44.18002034121909
-----
0.11158361742220538
44.28686915151622
-----
0.12903186174639994
44.39845276893843
-----
0.06105950199823853
44.527484630684825
-----
0.06126845147209275
44.58854413268306
-----
0.10649693942896589
44.64981258415516
-----
0.10501788763488946
44.75630952358412
-----
0.05622987668931128
44.86132741121901
-----
0.11341154398141026
44.917557287908316
-----
0.09748987924485912
45.03096883188972
-----
0.056801834504994805
45.128458711134584
-----
0.05900781033467038
45.18526054563958
-----
0.10297736066665167
45.24426835597425
-----
0.0561826568973926
45.3472457166409
-----
0.10023072902961781
45.40342837353829
-----
0.09446841518762926
45.50365910256791
-----
0.12262504707744595
45.598127517755536
-----
0.09979268406060357
45.720752564832985
-----
0.11108379020783166
45.820545

0.07189354985216716
61.7733081053994
-----
0.07649201852375938
61.84520165525157
-----
0.07470693135849789
61.92169367377533
-----
0.0363578290800822
61.99640060513383
-----
0.09187164962949758
62.03275843421391
-----
0.038060711644920454
62.12463008384341
-----
0.07350243784978301
62.162690795488324
-----
0.07271974473209286
62.236193233338106
-----
0.07457210182790323
62.3089129780702
-----
0.09172784214494926
62.3834850798981
-----
0.08158255677573634
62.47521292204305
-----
0.04094733720695875
62.556795478818785
-----
0.0812981751994426
62.59774281602574
-----
0.07055184955994719
62.679040991225186
-----
0.08197883289670307
62.749592840785134
-----
0.07500262035612122
62.831571673681836
-----
0.07541782058124406
62.90657429403796
-----
0.07960311513694973
62.9819921146192
-----
0.06870443321689619
63.06159522975615
-----
0.08001917933819297
63.130299662973044
-----
0.09240769162970254
63.21031884231124
-----
0.09030309598930605
63.30272653394094
-----
0.0724216273272991
63.39302962

0.07695780400455332
76.06404640781116
-----
0.032270305963141774
76.14100421181571
-----
0.06010041343627819
76.17327451777885
-----
0.06667943648521726
76.23337493121512
-----
0.06565492830003823
76.30005436770034
-----
0.07528104174319167
76.36570929600038
-----
0.06122930002686562
76.44099033774357
-----
0.06156747264666737
76.50221963777044
-----
0.0642055142755617
76.56378711041711
-----
0.0609183445478158
76.62799262469267
-----
0.06285929292487628
76.68891096924048
-----
0.03446118098626925
76.75177026216535
-----
0.05945659672589626
76.78623144315162
-----
0.05997360111591018
76.84568803987752
-----
0.05840782698588979
76.90566164099343
-----
0.06676392717487936
76.96406946797931
-----
0.06179835816865783
77.03083339515419
-----
0.06046094243951361
77.09263175332285
-----
0.06730379877525272
77.15309269576237
-----
0.0766082976145042
77.22039649453762
-----
0.06662224412914158
77.29700479215212
-----
0.06702296338906372
77.36362703628126
-----
0.059232993406844334
77.4306499996

0.05391092489833675
86.81203076129573
-----
0.054032396655415864
86.86594168619406
-----
0.053631463193485024
86.91997408284948
-----
0.0326776382955432
86.97360554604296
-----
0.05301913865591659
87.0062831843385
-----
0.053257523272462526
87.05930232299441
-----
0.0576001325002875
87.11255984626688
-----
0.06674434229988634
87.17015997876716
-----
0.05250732371721714
87.23690432106704
-----
0.055055397130483665
87.28941164478427
-----
0.02988743966909704
87.34446704191475
-----
0.060171207514836005
87.37435448158385
-----
0.05365680595135636
87.43452568909869
-----
0.051711099222036795
87.48818249505005
-----
0.06599513294373341
87.53989359427209
-----
0.057486374588455916
87.60588872721583
-----
0.052961681244054135
87.66337510180428
-----
0.05780005421980708
87.71633678304833
-----
0.05761355865109327
87.77413683726814
-----
0.054422018194390055
87.83175039591924
-----
0.057490894646379076
87.88617241411363
-----
0.05970066673518012
87.94366330876001
-----
0.0640922805593888
88.003

...

308.2471437112824
-----
0.006932710466631178
308.2572374399821
-----
0.00933773768634498
308.2641701504487
-----
0.008218541910108198
308.27350788813504
-----
0.010268760352346687
308.2817264300451
-----
0.008579695637939486
308.2919951903975
-----
0.007960403249597518
308.3005748860354
-----
0.01047562988881298
308.308535289285
-----
0.011012772611095819
308.31901091917376
-----
0.009837330503759269
308.33002369178485
-----
0.011799966598102896
308.33986102228863
-----
0.010631318949905016
308.35166098888675
-----
0.011118473608190398
308.36229230783664
-----
0.009836796526914621
308.3734107814448
-----
0.011642811487987088
308.38324757797176
-----
0.012012215555842096
308.39489038945976
-----
0.010516027698056013
308.4069026050156
-----
0.010876795301604536
308.41741863271363
-----
0.011883195432829771
308.42829542801525
-----
0.012290314115931907
308.44017862344805
-----
0.010848593124650033
308.452468937564
-----
0.01227239382864495
308.46331753068864
-----
0.012259905098581613
308

0.013639917085461485
311.5223287052791
-----
0.013563899099440384
311.5359686223646
-----
0.01284718092068403
311.549532521464
-----
0.01291320951733632
311.5623797023847
-----
0.012803338672995187
311.57529291190207
-----
0.012889878559836932
311.58809625057506
-----
0.012724314955166391
311.6009861291349
-----
0.0124587159136783
311.6137104440901
-----
0.012080268798772569
311.62616916000377
-----
0.012201646486132176
311.63824942880257
-----
0.01280395367385307
311.6504510752887
-----
0.012269524517201839
311.66325502896257
-----
0.011358050330281413
311.67552455347976
-----
0.013047230930991445
311.68688260381003
-----
0.011929003641215966
311.699929834741
-----
0.012763565179633273
311.7118588383822
-----
0.011630135628902132
311.72462240356185
-----
0.012064354619191631
311.73625253919073
-----
0.012082648647174447
311.74831689380994
-----
0.011455291576527407
311.7603995424571
-----
0.013053477478868019
311.7718548340336
-----
0.011605379160436935
311.7849083115125
-----
0.01307

0.017370810929351232
314.7841357999086
-----
0.017544726681677125
314.801506610838
-----
0.017543748921808315
314.8190513375196
-----
0.01993931514146316
314.83659508644143
-----
0.017394028475431856
314.8565344015829
-----
0.017513306918053265
314.87392843005836
-----
0.017058519547154496
314.8914417369764
-----
0.0175460896985385
314.90850025652355
-----
0.019567619863978078
314.9260463462221
-----
0.018538627026009488
314.9456139660861
-----
0.0209395523348545
314.96415259311215
-----
0.018746390879773447
314.985092145447
-----
0.020944622813618096
315.0038385363268
-----
0.018610234593686145
315.0247831591404
-----
0.017960361976202772
315.04339339373405
-----
0.01885250748088575
315.06135375571023
-----
0.019265724803615002
315.0802062631911
-----
0.021219771301224915
315.0994719879947
-----
0.019059714171580363
315.12069175929594
-----
0.019613012128866073
315.1397514734675
-----
0.02119246656485339
315.15936448559637
-----
0.01862482685448634
315.1805569521612
-----
0.0191107521

0.019581440759579277
317.9682928145299
-----
0.018742976704534644
317.98787425528946
-----
0.019801804558419938
318.00661723199397
-----
0.01889741479567519
318.0264190365524
-----
0.01916518264015906
318.04531645134807
-----
0.02041099704355616
318.06448163398824
-----
0.01969948766485359
318.0848926310318
-----
0.020038228667453216
318.10459211869664
-----
0.01894097827272969
318.1246303473641
-----
0.019050492234569533
318.1435713256368
-----
0.019599241746430095
318.16262181787135
-----
0.018955036586618
318.1822210596178
-----
0.019303730952034002
318.2011760962044
-----
0.01925937982443625
318.22047982715645
-----
0.019056102717183575
318.2397392069809
-----
0.019430789628656006
318.2587953096981
-----
0.01968523424145667
318.27822609932673
-----
0.019383605936480527
318.2979113335682
-----
0.01889555170000818
318.3172949395047
-----
0.01966020924154038
318.33619049120466
-----
0.01959935994392115
318.3558507004462
-----
0.01966044247709143
318.3754500603901
-----
0.0194585780601

IndexError: deque index out of range

In [141]:
[l.value for l in dqn.replay_memory.sumTree.leaf_nodes]

[0,
 0.31808150778174005,
 7.831204289189859,
 0.6893852388674514,
 0.2859036435119703,
 0.5610784079869724,
 0.4836863297714358,
 0.5722539581112602,
 0.44370504036477665,
 0.5311151552026265,
 0.49874618346113536,
 0.42130060043273265,
 0.3355760084232688,
 0.41233072361424034,
 0.34123425157959264,
 0.4122208014159609,
 0.3143489086045308,
 0.30906830208946995,
 0.3022898364154292,
 0.3070163396510295,
 0.3030469150636286,
 0.2888295489006427,
 0.16812095810086708,
 0.3546644077443775,
 0.34569800843088977,
 0.28327161253077904,
 0.3007194498365242,
 0.2953991371960584,
 0.2698004606917925,
 0.14248948422278238,
 0.14502244385952365,
 0.14250957260767946,
 0.25615873442126735,
 0.23089691103722318,
 0.2625206517382457,
 0.23687484950004534,
 0.2390872541672533,
 0.23479198999300052,
 0.22170047829308398,
 0.12011414107959087,
 0.2486975737916927,
 0.22944785935990997,
 0.2277681122991334,
 0.250922444471039,
 0.25180481308226754,
 0.24613967631019426,
 0.23753091566360351,
 0.193178

In [198]:
x = []
for i in range(10000000):
    x.append(dqn.replay_memory.sumTree.draw_idx())
sum([1 for i in x if i == 2]) / sum([1 for j in x if j == 1])
#26, passt! (schau drueber was richtig)

KeyboardInterrupt: 

In [196]:
len([l.value for l in dqn.replay_memory.sumTree.leaf_nodes])

100000

In [127]:
dqn.replay_memory.sumTree.top_node.value

tensor([inf], device='cuda:0')

In [130]:
dqn.replay_memory.sumTree.top_node.value.item()

inf

In [111]:
q_vals = torch.tensor([[ 9.4305,  5.4226,  8.4617,  6.7611,  8.7324, 17.9572]])
action_ = torch.tensor([env.action_space.sample()])
q_vals.gather(1, action_.unsqueeze(-1)).squeeze(-1)

tensor([6.7611])

In [108]:
action

tensor([5])

In [109]:
action_

tensor(5)